[View in Colaboratory](https://colab.research.google.com/github/monimoyd/DNST_CIFAR10/blob/master/DNST_CIFAR10_monimoy_changes_1.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.
tensorflow-gpu 1.9.0 has requirement tensorboard<1.10.0,>=1.9.0, but you'll have tensorboard 1.10.0 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
E:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.optimizers import SGD

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 250
l = 22
num_filter = 16
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Rotate images by 45 degrees
datagen1 = ImageDataGenerator(rotation_range=45)

# fit parameters from data
datagen1.fit(x_train)

# Get 1000 augmented images and append to the training data
for x_rotated_images_batch, y_rotated_images_batch in datagen1.flow(x_train, y_train, batch_size=1000):
    x_train = np.append(x_train, x_rotated_images_batch, axis=0)
    y_train = np.append(y_train, y_rotated_images_batch, axis=0)
    break

# Horizontally flip image
datagen2 = ImageDataGenerator(horizontal_flip=True)

# fit parameters from data
datagen2.fit(x_train)  

# Get 1000 augmented horizontally flipped images and append to the training data
for x_flipped_images_batch, y_flipped_images_batch in datagen2.flow(x_train, y_train, batch_size=1000):
    x_train = np.append(x_train, x_flipped_images_batch, axis=0)
    y_train = np.append(y_train, y_flipped_images_batch, axis=0)
    break


In [7]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [8]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return AvgPooling

In [9]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [10]:
input = Input(shape=(img_height, img_width, channel,))
layer1_transition = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

layer2_block = add_denseblock(layer1_transition, num_filter, dropout_rate)
layer2_transition = add_transition(layer2_block, num_filter, dropout_rate)
skip_connection1 = layer2_transition

layer3_block = add_denseblock(layer2_transition, num_filter, dropout_rate)
layer3_transition = add_transition(layer3_block, num_filter, dropout_rate)

layer4_block = add_denseblock(layer3_transition, num_filter, dropout_rate)
layer4_transition = add_transition(layer4_block, num_filter, dropout_rate)

layer5_skip_connection_block = add_denseblock(skip_connection1, num_filter, dropout_rate)
layer5_skip_connection_block = Conv2D(int(num_filter*compression), (5,5), use_bias=False)(layer5_skip_connection_block)
layer5_skip_connection_block = Conv2D(int(num_filter*compression), (5,5), use_bias=False)(layer5_skip_connection_block)
layer5_skip_connection_block = Conv2D(int(num_filter*compression), (5,5), use_bias=False)(layer5_skip_connection_block)
layer5_block = Concatenate(axis=-1)([layer5_skip_connection_block, layer4_transition])
layer5_transition = add_transition(layer5_block, num_filter, dropout_rate)

layer6_block = add_denseblock(layer5_transition,  num_filter, dropout_rate)
output = output_layer(layer6_block)

In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [12]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

## Callbacks For Early Stopping and Saving the Best Model

In [13]:
from keras.callbacks import Callback
class EarlyStoppingByValidationAccuracy(Callback):
    def __init__(self, monitor='val_acc', value=1.0, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        accuracy_value= logs.get(self.monitor)
       
        if accuracy_value >= self.value:
            self.model.stop_training = True
            if self.verbose == 1:
                print("Epoch %d: Threshold for early stopping has reached" % (epoch + 1))   

In [ ]:
from keras.callbacks import ModelCheckpoint
callbacks_list=[]
model_save_path= "best_model-CIFAR10-monimoy-my_computer2.h5"
callbacks_list.append(EarlyStoppingByValidationAccuracy(monitor='val_acc',  value=0.9201, verbose=1))
callbacks_list.append(ModelCheckpoint(model_save_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max'))

In [ ]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Train on 52000 samples, validate on 10000 samples
Epoch 1/250
52000/52000 [==============================] - 327s 6ms/step - loss: 1.9162 - acc: 0.2640 - val_loss: 1.7352 - val_acc: 0.3564

Epoch 00001: val_acc improved from -inf to 0.35640, saving model to best_model-CIFAR10-monimoy-my_computer2.h5
Epoch 2/250
52000/52000 [==============================] - 298s 6ms/step - loss: 1.6155 - acc: 0.3910 - val_loss: 1.7642 - val_acc: 0.3890

Epoch 00002: val_acc improved from 0.35640 to 0.38900, saving model to best_model-CIFAR10-monimoy-my_computer2.h5
Epoch 3/250
52000/52000 [==============================] - 306s 6ms/step - loss: 1.4984 - acc: 0.4455 - val_loss: 1.4634 - val_acc: 0.4653

Epoch 00003: val_acc improved from 0.38900 to 0.46530, saving model to best_model-CIFAR10-monimoy-my_computer2.h5
Epoch 4/250
52000/52000 [==============================] - 304s 6ms/step - loss: 1.4156 - acc: 0.4786 - val_loss: 1.8466 - val_acc: 0.4037

Epoch 00004: val_acc did not improve from 0.46530
E

In [ ]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")